#Beginning----Başlangıç

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
veri_yolu = '/content/drive/My Drive/yapay_zeka/Phishing_Email.csv'
model_yolu = '/content/drive/My Drive/yapay_zeka/phishing_ai.joblib' #model yolu

#Libraries----Kütüphaneler

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN,Dense
from keras.utils import to_categorical
import joblib   #modelimizi kayıt etmek ve sonra kullanmak için
import tensorflow as tf


#Data Prossesing----------Veri İşlenmesi

In [ ]:
mail_verisi = pd.read_csv(veri_yolu)  #veriyi okuma(pandas kütüphanesi kullanıldı)
mail_verisi = mail_verisi.where((pd.notnull(mail_verisi)),'') #mail verisi boşsa kaldıralım

mail_verisi.loc[mail_verisi['Email Type']== 'Safe Email','Email Type']=0        #güvenli mailse 0 atadık
mail_verisi.loc[mail_verisi['Email Type']== 'Phishing Email','Email Type']=1    #güvenli olmayan mailse 1 atadık

X = mail_verisi['Email Text']
Y = mail_verisi['Email Type']
Y = to_categorical(Y)

#Tokenizer

In [ ]:
tokenizer = Tokenizer()  #tokenizer tanımlama
tokenizer.fit_on_texts(X)  #text verileri tokenizere fit edildi
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index  #word index eklendi
max_sequence_length = 100
veri = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
#verileri sırasıyla görmek için for içine yazdık
for seq in word_index:
  print(seq)

#Test Train Data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(veri, Y, test_size=0.2, random_state = 3)

#Create a model --------Model Oluşturma

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index)+1,128,input_length=max_sequence_length))
model.add(SimpleRNN(64))  #64 nöron
model.add(Dense(2,activation='softmax')) #birden fazla çıkış için softmax kullandık

#Model Compile--------Model Derleme

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'] ) #metrics:doğrulama aşaması

#Train----------Eğitim Süreci

In [ ]:
model.fit(x_train,y_train, epochs=5,batch_size = 25, validation_data=(x_test,y_test))
 #batch_size:Her adımda kaçar tane alsın

Epoch 1/5
597/597 [==============================] - 320s 533ms/step - loss: 0.2537 - accuracy: 0.8868 - val_loss: 0.1397 - val_accuracy: 0.9469
Epoch 2/5
597/597 [==============================] - 317s 531ms/step - loss: 0.0499 - accuracy: 0.9777 - val_loss: 0.1172 - val_accuracy: 0.9582
Epoch 3/5
597/597 [==============================] - 331s 555ms/step - loss: 0.0346 - accuracy: 0.9832 - val_loss: 0.1307 - val_accuracy: 0.9568
Epoch 4/5
597/597 [==============================] - 318s 533ms/step - loss: 0.0293 - accuracy: 0.9857 - val_loss: 0.1221 - val_accuracy: 0.9582
Epoch 5/5
597/597 [==============================] - 330s 553ms/step - loss: 0.0427 - accuracy: 0.9809 - val_loss: 0.1789 - val_accuracy: 0.9416


#Model Save-----Model Kayıt

In [ ]:
model.save(model_yolu)

#Model evaluation--------Model Değerlendirme

In [ ]:
loss,accuracy = model.evaluate(x_test,y_test)
print('Test verilerin doğruluğu :',accuracy)

117/117 [==============================] - 1s 8ms/step - loss: 0.1789 - accuracy: 0.9416
Test verilerin doğruluğu : 0.9415549635887146


#Real Time Test-------Gerçek Zamanlı Test

#Model Loading-----Model Yüklenmesi

In [ ]:
model = tf.keras.models.load_model(model_yolu)

#Test Phase----Test Aşaması

In [ ]:
while True:
  input_mail = input("E-posta mesajını giriniz :")
  input_sequence = tokenizer.texts_to_sequences([input_mail])  #girdiği veri üzerinde tokenizer işlemi yapacağız.
  input_data = pad_sequences(input_sequence,maxlen=max_sequence_length)
  tahmin = model.predict(input_data)

  if input_mail=='quit':
    break
  elif np.argmax(tahmin) ==1:
    print("Bu bir kimlik avı e-postası olabilir.")
  else:
    print("Bu güvenli bir e-postadır.")
  oran = tahmin[0][np.argmax(tahmin)]
  print("oran:",np.argmax(tahmin))
  print("Eminlik oranı :",oran)

E-posta mesajını giriniz :hello
1/1 [==============================] - 0s 36ms/step
Bu bir kimlik avı e-postası olabilir.
oran: 1
Eminlik oranı : 0.75806946
E-posta mesajını giriniz :hi
1/1 [==============================] - 0s 23ms/step
Bu güvenli bir e-postadır.
oran: 0
Eminlik oranı : 0.6901549


KeyboardInterrupt: Interrupted by user